## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有8種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probability = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

```
觀察狀態 = 'dry', 'dry', 'dry'
Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104
Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888

最大機率為: Sunny, Sunny, Sunny
```

![prob](prob.png)

In [1]:
# 初始＊發射＊轉移＊發射＊轉移＊發射
# Sunny, Rainy, Sunny:
SRS = 0.4*(0.6)*0.4*(0.1)*0.3*(0.6)
print('SRS = ', SRS)
# Sunny, Rainy, Rainy:
SRR = 0.4*(0.6)*0.4*(0.1)*0.7*(0.1)
print('SRR = ', SRR)
# Sunny, Sunny, Rainy:
SSR = 0.4*(0.6)*0.6*(0.6)*0.4*(0.1)
print('SSR = ', SSR)

SRS =  0.0017280000000000002
SRR =  0.0006720000000000001
SSR =  0.0034560000000000003


In [2]:
# Rainy, Rainy, Sunny:
RRS = 0.6*(0.1)*0.7*(0.1)*0.3*(0.6)
print('RRS = ', RRS)
# Rainy, Rainy, Rainy:
RRR = 0.6*(0.1)*0.7*(0.1)*0.7*(0.1)
print('RRR = ', RRR)
# Rainy, Sunny, Rainy:
RSR = 0.6*(0.1)*0.3*(0.6)*0.4*(0.1)
print('RSR = ', RSR)

RRS =  0.0007559999999999998
RRR =  0.000294
RSR =  0.00043200000000000004


### 根據上述條件, 寫出Viterbi應用程式

In [263]:
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry
states = ('sunny', 'rainy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probability = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

In [4]:
V = [{}]
path = {}
for y in states:
    # 計算初始x發射機率
    V[0][y] = start_probability[y] * emission_probability[y][observations[0]]
    # 將初始狀況加進 path
    path[y] = [y]
    
for t in range(1, len(observations)): #跑後續所有觀察狀況
    V.append({})
    newpath = {}
    # 計算後續機率
    for cur_state in states: # 現在狀況
        for pre_state in states: #前一個狀況
            pre_V = V[t-1][pre_state] #前一個機率
            trans_p = transition_probability[pre_state][cur_state]
            emit_p = emission_probability[cur_state][observations[t]]
            
            V[t][cur_state] = max([(pre_V * trans_p * emit_p)])
            state = pre_state
            print(V[t][cur_state])
            print(state, '\n')

            newpath[cur_state] = path[state] + [cur_state]
        path = newpath
(prob, state) = max([(V[len(observations) - 1][final_state], final_state) for final_state in states])

print(V, '\n')

0.08639999999999999
sunny 

0.010799999999999999
rainy 

0.009600000000000001
sunny 

0.0042
rainy 

0.003887999999999999
sunny 

0.0007559999999999998
rainy 

0.00043200000000000004
sunny 

0.000294
rainy 

[{'sunny': 0.24, 'rainy': 0.06}, {'sunny': 0.010799999999999999, 'rainy': 0.0042}, {'sunny': 0.0007559999999999998, 'rainy': 0.000294}] 



In [431]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for y in states:
        # 計算初始*發射機率
        V[0][y] = start_p[y] * emit_p[y][obs[0]]
        # 將初始狀況加進 paththenme
        path[y] = [y]

        
    # Run Viterbi for t > 0
    for t in range(1,len(obs)):
        V.append({})
        newpath = {}
        
        # 計算後續機率
        for cur_state in states:
            (prob, state) = max([(
                V[t-1][pre_state] * trans_p[pre_state][cur_state] * emit_p[cur_state][obs[t]],
                pre_state)
                for pre_state in states
            ])
            
            V[t][cur_state] = prob
            newpath[cur_state] = path[state] + [cur_state]
        
        path = newpath
        print('\n抵達第 {} 節點最可能路徑為 {}'.format((str(t+1)), [i for i in newpath.values()]))

    (prob, state) = max([(V[len(obs) - 1][final_state], final_state) for final_state in states])
    print('\n最終結果:\n>>> 最可能路徑: {}\n>>> 機率: {}'.format(path[state], round(prob, 5)))
    
    return (round(prob, 5), path[state])

In [432]:
observations = ('dry', 'dry', 'dry')
result = viterbi(observations,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probability)


抵達第 2 節點最可能路徑為 [['sunny', 'sunny'], ['sunny', 'rainy']]

抵達第 3 節點最可能路徑為 [['sunny', 'sunny', 'sunny'], ['sunny', 'sunny', 'rainy']]

最終結果:
>>> 最可能路徑: ['sunny', 'sunny', 'sunny']
>>> 機率: 0.0311


In [433]:
observations = ('dry', 'damp', 'damp')
result = viterbi(observations,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probability)


抵達第 2 節點最可能路徑為 [['sunny', 'sunny'], ['sunny', 'rainy']]

抵達第 3 節點最可能路徑為 [['sunny', 'rainy', 'sunny'], ['sunny', 'rainy', 'rainy']]

最終結果:
>>> 最可能路徑: ['sunny', 'rainy', 'rainy']
>>> 機率: 0.02419
